*2018.06.26*

* TensorFlow Eager Execution について
* 通常モードのTensorFlow
* EagerモードのTensorFlow

---

**TensorFlow Eager Execution について**

---

* 2018年1月26日（米国時間）, Googleがオープンソース機械学習ライブラリの最新版「TensorFlow 1.5」を公開.
    * 「Eager Execution for TensorFlow」
        * Eager Execution for TensorFlowは, 「Define by Run」型のプログラミングスタイルを可能にするインタフェースであり, これを有効にすると, PythonからTensorFlow演算を呼び出してすぐに実行できるようになる.
            * Define by Run
                * 計算グラフ（ニューラルネットの構造）の構築をデータを流しながら行う.
            * Define and Run
                * 計算グラフを構築してから, そこにデータを流していく.
    * 「TensorFlow Lite」
        * モバイルや組み込みデバイス向けのTensorFlowの軽量版. 学習済みのTensorFlowモデルを「.tflite」ファイルに変換し, モバイルデバイスを使って低レイテンシで実行できる.
    * GPUアクセラレーション対応の強化
        * 新たに「CUDA 9」と「cuDNN 7」をサポート.
    
    
* GoogleはEager Execution for TensorFlowのメリットとして, 下記を挙げている.

    * 実行時エラーの即時確認と, Pythonツールと統合された迅速なデバッグ
    * 使いやすいPython制御フローを利用した動的モデルのサポート
    * カスタムおよび高次勾配の強力なサポート
    * ほとんどのTensorFlow演算が利用可能
    
---

**通常モードのTensorFlow**

In [5]:
import sys, os
import numpy as np
from sklearn import datasets, model_selection
from tqdm import tqdm

iris = datasets.load_iris()
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(iris.data, iris.target)

train_x.shape, train_y.shape, valid_x.shape, valid_y.shape

((112, 4), (112,), (38, 4), (38,))

In [3]:
import tensorflow as tf

In [4]:
# フローの定義

input_size = 4
output_size = 3
hidden_size = 20

x_ph = tf.placeholder(tf.float32, shape=[None, input_size])
y_ph = tf.placeholder(tf.int32, [None])
y_oh = tf.one_hot(y_ph, depth=output_size, dtype=tf.float32)

fc1_w = tf.Variable(tf.truncated_normal([input_size, hidden_size], stddev=0.1), dtype=tf.float32)
fc1_b = tf.Variable(tf.constant(0.1, shape=[hidden_size]), dtype=tf.float32)
fc1 = tf.nn.relu(tf.matmul(x_ph, fc1_w) + fc1_b)

fc2_w = tf.Variable(tf.truncated_normal([hidden_size, hidden_size], stddev=0.1), dtype=tf.float32)
fc2_b = tf.Variable(tf.constant(0.1, shape=[hidden_size]), dtype=tf.float32)
fc2 = tf.nn.relu(tf.matmul(fc1, fc2_w) + fc2_b)

fc3_w = tf.Variable(tf.truncated_normal([hidden_size, output_size], stddev=0.1), dtype=tf.float32)
fc3_b = tf.Variable(tf.constant(0.1, shape=[output_size]), dtype=tf.float32)
y_pre = tf.matmul(fc2, fc3_w) + fc3_b

cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_oh, logits=y_pre))
train_step = tf.train.AdamOptimizer().minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_pre, 1), tf.argmax(y_oh, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# 学習

epoch_num = 30
batch_size = 16

init = tf.global_variables_initializer()

with tf.Session() as sess:
    
    sess.run(init)

    for epoch in tqdm(range(epoch_num), file=sys.stdout):
        
        perm = np.random.permutation(len(train_x))
        
        for i in range(0, len(train_x), batch_size):
            
            batch_x = train_x[perm[i:i+batch_size]]
            batch_y = train_y[perm[i:i+batch_size]]
            train_step.run(feed_dict={x_ph: batch_x, y_ph: batch_y})
            
        train_loss = cross_entropy.eval(feed_dict={x_ph: train_x, y_ph: train_y})
        valid_loss = cross_entropy.eval(feed_dict={x_ph: valid_x, y_ph: valid_y})
        train_acc = accuracy.eval(feed_dict={x_ph: train_x, y_ph: train_y})
        valid_acc = accuracy.eval(feed_dict={x_ph: valid_x, y_ph: valid_y})
        
        if (epoch+1)%5 == 0:
            tqdm.write('epoch:\t{}\ttrain/loss:\t{:.5f}\tvalid/loss:\t{:.5f}\ttrain/accuracy:\t{:.5f}\tvalid/accuracy:\t{:.5f}'.format(epoch+1, train_loss, valid_loss, train_acc, valid_acc))

epoch:	5	train/loss:	1.02495	valid/loss:	1.01906	train/accuracy:	0.33036	valid/accuracy:	0.34211
epoch:	10	train/loss:	0.91124	valid/loss:	0.89727	train/accuracy:	0.65179	valid/accuracy:	0.71053
epoch:	15	train/loss:	0.75381	valid/loss:	0.72561	train/accuracy:	0.65179	valid/accuracy:	0.71053
epoch:	20	train/loss:	0.59629	valid/loss:	0.55483	train/accuracy:	0.65179	valid/accuracy:	0.71053
epoch:	25	train/loss:	0.48327	valid/loss:	0.44436	train/accuracy:	0.86607	valid/accuracy:	0.84211
epoch:	30	train/loss:	0.40163	valid/loss:	0.36836	train/accuracy:	0.95536	valid/accuracy:	0.92105
100%|██████████| 30/30 [00:00<00:00, 42.93it/s]


**EagerモードのTensorFlow**

---

* Eagerモードを実行するには, 以下のように初期化を行うが, 一度通常モードで実行してしまうとEagerモードを実行できない.

In [5]:
import tensorflow as tf
import tensorflow.contrib.eager as tfe

tf.enable_eager_execution()

print("TensorFlow version: {}".format(tf.VERSION))
print("Eager execution: {}".format(tf.executing_eagerly()))

ValueError: tf.enable_eager_execution must be called at program startup.

* Jupyterで実行する場合には一度カーネルを再起動する.

In [1]:
import tensorflow as tf
import tensorflow.contrib.eager as tfe

tf.enable_eager_execution()

print("TensorFlow version: {}".format(tf.VERSION))
print("Eager execution: {}".format(tf.executing_eagerly()))

/usr/local/lib/python3.6/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


TensorFlow version: 1.8.0
Eager execution: True


* 逆も然り. （以下はさっきのコード）

In [2]:
# フローの定義

input_size = 4
output_size = 3
hidden_size = 20

x_ph = tf.placeholder(tf.float32, shape=[None, input_size])
y_ph = tf.placeholder(tf.int32, [None])
y_oh = tf.one_hot(y_ph, depth=output_size, dtype=tf.float32)

fc1_w = tf.Variable(tf.truncated_normal([input_size, hidden_size], stddev=0.1), dtype=tf.float32)
fc1_b = tf.Variable(tf.constant(0.1, shape=[hidden_size]), dtype=tf.float32)
fc1 = tf.nn.relu(tf.matmul(x_ph, fc1_w) + fc1_b)

fc2_w = tf.Variable(tf.truncated_normal([hidden_size, hidden_size], stddev=0.1), dtype=tf.float32)
fc2_b = tf.Variable(tf.constant(0.1, shape=[hidden_size]), dtype=tf.float32)
fc2 = tf.nn.relu(tf.matmul(fc1, fc2_w) + fc2_b)

fc3_w = tf.Variable(tf.truncated_normal([hidden_size, output_size], stddev=0.1), dtype=tf.float32)
fc3_b = tf.Variable(tf.constant(0.1, shape=[output_size]), dtype=tf.float32)
y_pre = tf.matmul(fc2, fc3_w) + fc3_b

cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_oh, logits=y_pre))
train_step = tf.train.AdamOptimizer().minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_pre, 1), tf.argmax(y_oh, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# 学習

epoch_num = 30
batch_size = 16

init = tf.global_variables_initializer()

with tf.Session() as sess:
    
    sess.run(init)

    for epoch in tqdm(range(epoch_num), file=sys.stdout):
        
        perm = np.random.permutation(len(train_x))
        
        for i in range(0, len(train_x), batch_size):
            
            batch_x = train_x[perm[i:i+batch_size]]
            batch_y = train_y[perm[i:i+batch_size]]
            train_step.run(feed_dict={x_ph: batch_x, y_ph: batch_y})
            
        train_loss = cross_entropy.eval(feed_dict={x_ph: train_x, y_ph: train_y})
        valid_loss = cross_entropy.eval(feed_dict={x_ph: valid_x, y_ph: valid_y})
        train_acc = accuracy.eval(feed_dict={x_ph: train_x, y_ph: train_y})
        valid_acc = accuracy.eval(feed_dict={x_ph: valid_x, y_ph: valid_y})
        
        if (epoch+1)%5 == 0:
            tqdm.write('epoch:\t{}\ttrain/loss:\t{:.5f}\tvalid/loss:\t{:.5f}\ttrain/accuracy:\t{:.5f}\tvalid/accuracy:\t{:.5f}'.format(epoch+1, train_loss, valid_loss, train_acc, valid_acc))

RuntimeError: tf.placeholder() is not compatible with eager execution.

* Eagerモードでアイリスの問題をやってみる.
* 公式ドキュメントのサンプルコード.
    * https://www.tensorflow.org/get_started/eager

In [4]:
import os

train_dataset_url = "http://download.tensorflow.org/data/iris_training.csv"

train_dataset_fp = tf.keras.utils.get_file(fname=os.path.basename(train_dataset_url), origin=train_dataset_url)

def parse_csv(line):
    example_defaults = [[0.], [0.], [0.], [0.], [0]]  # sets field types
    parsed_line = tf.decode_csv(line, example_defaults)
    # First 4 fields are features, combine into single tensor
    features = tf.reshape(parsed_line[:-1], shape=(4,))
    # Last field is the label
    label = tf.reshape(parsed_line[-1], shape=())
    return features, label

train_dataset = tf.data.TextLineDataset(train_dataset_fp)
train_dataset = train_dataset.skip(1)             # skip the first header row
train_dataset = train_dataset.map(parse_csv)      # parse each row
train_dataset = train_dataset.shuffle(buffer_size=1000)  # randomize
train_dataset = train_dataset.batch(32)

model = tf.keras.Sequential([
    tf.keras.layers.Dense(10, activation="relu", input_shape=(4,)),  # input shape required
    tf.keras.layers.Dense(10, activation="relu"),
    tf.keras.layers.Dense(3)
])

def loss(model, x, y):
    y_ = model(x)
    return tf.losses.sparse_softmax_cross_entropy(labels=y, logits=y_)

def grad(model, inputs, targets):
    with tf.GradientTape() as tape:
        loss_value = loss(model, inputs, targets)
    return tape.gradient(loss_value, model.variables)

optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)

train_loss_results = []
train_accuracy_results = []

num_epochs = 201

for epoch in range(num_epochs):
    epoch_loss_avg = tfe.metrics.Mean()
    epoch_accuracy = tfe.metrics.Accuracy()

    # Training loop - using batches of 32
    for x, y in train_dataset:
        # Optimize the model
        grads = grad(model, x, y)
        optimizer.apply_gradients(zip(grads, model.variables), global_step=tf.train.get_or_create_global_step())

        # Track progress
        epoch_loss_avg(loss(model, x, y))  # add current batch loss
        # compare predicted label to actual label
        epoch_accuracy(tf.argmax(model(x), axis=1, output_type=tf.int32), y)

    # end epoch
    train_loss_results.append(epoch_loss_avg.result())
    train_accuracy_results.append(epoch_accuracy.result())

    if epoch % 50 == 0:
        print("Epoch {:03d}: Loss: {:.3f}, Accuracy: {:.3%}".format(epoch, epoch_loss_avg.result(), epoch_accuracy.result()))

8192/2194 [================================================================================================================] - 0s 0us/step
Epoch 000: Loss: 1.157, Accuracy: 30.000%
Epoch 050: Loss: 0.829, Accuracy: 70.000%
Epoch 100: Loss: 0.639, Accuracy: 70.000%
Epoch 150: Loss: 0.529, Accuracy: 85.833%
Epoch 200: Loss: 0.430, Accuracy: 93.333%


* TensorFlowの便利？な関数がめっちゃ使われてて分かりづらい.
* tf.data はTensorFlowだけの世界で閉じることができるので, 別途使ってみると便利かも？
* 上記のコードを参考に, scikit-learnから落としたデータ（numpy）からスタートにして, 少し書き直してみた.

In [6]:
train_x_tf = tf.convert_to_tensor(train_x, dtype=tf.float32)
train_y_tf = tf.convert_to_tensor(train_y, dtype=tf.int32)
valid_x_tf = tf.convert_to_tensor(valid_x, dtype=tf.float32)
valid_y_tf = tf.convert_to_tensor(valid_y, dtype=tf.int32)

model = tf.keras.Sequential([
    tf.keras.layers.Dense(20, activation='relu', input_shape=(4,)),
    tf.keras.layers.Dense(20, activation='relu'),
    tf.keras.layers.Dense(3)
])

def lossfun(model, x, y):
    y_pre = model(x)
    y_oh = tf.one_hot(y, depth=output_size, dtype=tf.float32)
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_oh, logits=y_pre))
    return cross_entropy

def grad(model, x, y):
    with tf.GradientTape() as tape:
        loss = lossfun(model, x, y)
    return tape.gradient(loss, model.variables)

epoch_num = 30
batch_size = 16

optimizer = tf.train.AdamOptimizer()

for epoch in tqdm(range(epoch_num), file=sys.stdout):
    
    n, _ = train_x_tf.shape
    n = n.value
    perm = np.random.permutation(n)
    
    for i in range(0, n, batch_size):

        batch_x = tf.gather(train_x_tf, perm[i:i+batch_size])
        batch_y = tf.gather(train_y_tf, perm[i:i+batch_size])

        grads = grad(model, batch_x, batch_y)
        optimizer.apply_gradients(zip(grads, model.variables), global_step=tf.train.get_or_create_global_step())

    train_loss = lossfun(model, train_x_tf, train_y_tf)
    correct_prediction = tf.equal(tf.argmax(model(train_x_tf), axis=1, output_type=tf.int32), train_y_tf)
    train_acc = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    valid_loss = lossfun(model, valid_x_tf, valid_y_tf)
    correct_prediction = tf.equal(tf.argmax(model(valid_x_tf), axis=1, output_type=tf.int32), valid_y_tf)
    valid_acc = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        
    if (epoch+1)%5 == 0:
        tqdm.write('epoch:\t{}\ttrain/loss:\t{:.5f}\tvalid/loss:\t{:.5f}\ttrain/accuracy:\t{:.5f}\tvalid/accuracy:\t{:.5f}'.format(
            epoch+1, train_loss, valid_loss, train_acc, valid_acc)
                  )

epoch:	5	train/loss:	1.18120	valid/loss:	1.21846	train/accuracy:	0.26786	valid/accuracy:	0.23684
epoch:	10	train/loss:	0.96627	valid/loss:	1.00327	train/accuracy:	0.38393	valid/accuracy:	0.28947
epoch:	15	train/loss:	0.82718	valid/loss:	0.82987	train/accuracy:	0.66964	valid/accuracy:	0.71053
epoch:	20	train/loss:	0.67084	valid/loss:	0.65927	train/accuracy:	0.75000	valid/accuracy:	0.81579
epoch:	25	train/loss:	0.56404	valid/loss:	0.53081	train/accuracy:	0.86607	valid/accuracy:	0.89474
epoch:	30	train/loss:	0.50282	valid/loss:	0.46308	train/accuracy:	0.74107	valid/accuracy:	0.81579
100%|██████████| 30/30 [00:01<00:00, 16.45it/s]


* だいぶ分かってきた.
* 高レベルAPIの使い方をしているため, ちょっと生TensorFlowの書き方に寄せてみる.

In [7]:
"""
model = tf.keras.Sequential([
    tf.keras.layers.Dense(20, activation='relu', input_shape=(4,)),
    tf.keras.layers.Dense(20, activation='relu'),
    tf.keras.layers.Dense(3)
])
"""
class Model():
    
    def __init__(self):
        input_size = 4
        output_size = 3
        hidden_size = 20
        self.fc1_w = tfe.Variable(tf.truncated_normal([input_size, hidden_size], stddev=0.1), dtype=tf.float32)
        self.fc1_b = tfe.Variable(tf.constant(0.1, shape=[hidden_size]), dtype=tf.float32)
        self.fc2_w = tfe.Variable(tf.truncated_normal([hidden_size, hidden_size], stddev=0.1), dtype=tf.float32)
        self.fc2_b = tfe.Variable(tf.constant(0.1, shape=[hidden_size]), dtype=tf.float32)
        self.fc3_w = tfe.Variable(tf.truncated_normal([hidden_size, output_size], stddev=0.1), dtype=tf.float32)
        self.fc3_b = tfe.Variable(tf.constant(0.1, shape=[output_size]), dtype=tf.float32)
        self.variables = [
            self.fc1_w, self.fc1_b,
            self.fc2_w, self.fc2_b,
            self.fc3_w, self.fc3_b
        ]
        
    def __call__(self, x):
        h = tf.nn.relu(tf.matmul(x, self.fc1_w) + self.fc1_b)
        h = tf.nn.relu(tf.matmul(h, self.fc2_w) + self.fc2_b)
        y_pre = tf.matmul(h, self.fc3_w) + self.fc3_b
        return y_pre
    
model = Model()

def lossfun(model, x, y):
    y_pre = model(x)
    y_oh = tf.one_hot(y, depth=output_size, dtype=tf.float32)
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_oh, logits=y_pre))
    return cross_entropy

def grad(model, x, y):
    with tf.GradientTape() as tape:
        loss = lossfun(model, x, y)
    return tape.gradient(loss, model.variables)
    
train_x_tf = tf.convert_to_tensor(train_x, dtype=tf.float32)
train_y_tf = tf.convert_to_tensor(train_y, dtype=tf.int32)
valid_x_tf = tf.convert_to_tensor(valid_x, dtype=tf.float32)
valid_y_tf = tf.convert_to_tensor(valid_y, dtype=tf.int32)

epoch_num = 30
batch_size = 16

optimizer = tf.train.AdamOptimizer()

for epoch in tqdm(range(epoch_num), file=sys.stdout):
    
    n, _ = train_x_tf.shape
    n = n.value
    perm = np.random.permutation(n)
    
    for i in range(0, n, batch_size):

        batch_x = tf.gather(train_x_tf, perm[i:i+batch_size])
        batch_y = tf.gather(train_y_tf, perm[i:i+batch_size])

        grads = grad(model, batch_x, batch_y)
        optimizer.apply_gradients(zip(grads, model.variables), global_step=tf.train.get_or_create_global_step())

    train_loss = lossfun(model, train_x_tf, train_y_tf)
    correct_prediction = tf.equal(tf.argmax(model(train_x_tf), axis=1, output_type=tf.int32), train_y_tf)
    train_acc = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    valid_loss = lossfun(model, valid_x_tf, valid_y_tf)
    correct_prediction = tf.equal(tf.argmax(model(valid_x_tf), axis=1, output_type=tf.int32), valid_y_tf)
    valid_acc = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        
    if (epoch+1)%5 == 0:
        tqdm.write('epoch:\t{}\ttrain/loss:\t{:.5f}\tvalid/loss:\t{:.5f}\ttrain/accuracy:\t{:.5f}\tvalid/accuracy:\t{:.5f}'.format(
            epoch+1, train_loss, valid_loss, train_acc, valid_acc)
                  )

epoch:	5	train/loss:	1.05228	valid/loss:	1.08151	train/accuracy:	0.35714	valid/accuracy:	0.26316
epoch:	10	train/loss:	0.99127	valid/loss:	1.02852	train/accuracy:	0.35714	valid/accuracy:	0.26316
epoch:	15	train/loss:	0.86419	valid/loss:	0.88195	train/accuracy:	0.66071	valid/accuracy:	0.68421
epoch:	20	train/loss:	0.68577	valid/loss:	0.67165	train/accuracy:	0.66071	valid/accuracy:	0.68421
epoch:	25	train/loss:	0.53698	valid/loss:	0.49095	train/accuracy:	0.66071	valid/accuracy:	0.68421
epoch:	30	train/loss:	0.44538	valid/loss:	0.38509	train/accuracy:	0.83929	valid/accuracy:	0.84211
100%|██████████| 30/30 [00:01<00:00, 17.46it/s]


* 異なる点としては, 微分計算のところが通常モードとだいぶ様子が違う？
    * どうやらGradientTapeクラスで dloss/dw の微分計算をさせるらしい.
        * tape = tfe.GradientTape() として, tape(loss, w)とする. 
        
---

* kerasじゃない書き方（layers）だと, tfe.Networkクラスが便利そう.

In [8]:
class Model(tfe.Network):
    
    def __init__(self):
        super(Model, self).__init__()
        input_size = 4
        output_size = 3
        hidden_size = 20
        self.fc1 = self.track_layer(tf.layers.Dense(hidden_size, input_shape=(input_size, )))
        self.fc2 = self.track_layer(tf.layers.Dense(hidden_size, input_shape=(hidden_size, )))
        self.fc3 = self.track_layer(tf.layers.Dense(output_size, input_shape=(hidden_size, )))
    
    def __call__(self, x):
        h = tf.nn.relu(self.fc1(x))
        h = tf.nn.relu(self.fc2(h))
        y = self.fc3(h)
        return y
    
model = Model()

def lossfun(model, x, y):
    y_pre = model(x)
    y_oh = tf.one_hot(y, depth=output_size, dtype=tf.float32)
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_oh, logits=y_pre))
    return cross_entropy

def grad(model, x, y):
    with tf.GradientTape() as tape:
        loss = lossfun(model, x, y)
    return tape.gradient(loss, model.variables)
    
train_x_tf = tf.convert_to_tensor(train_x, dtype=tf.float32)
train_y_tf = tf.convert_to_tensor(train_y, dtype=tf.int32)
valid_x_tf = tf.convert_to_tensor(valid_x, dtype=tf.float32)
valid_y_tf = tf.convert_to_tensor(valid_y, dtype=tf.int32)

epoch_num = 30
batch_size = 16

optimizer = tf.train.AdamOptimizer()

for epoch in tqdm(range(epoch_num), file=sys.stdout):
    
    n, _ = train_x_tf.shape
    n = n.value
    perm = np.random.permutation(n)
    
    for i in range(0, n, batch_size):

        batch_x = tf.gather(train_x_tf, perm[i:i+batch_size])
        batch_y = tf.gather(train_y_tf, perm[i:i+batch_size])

        grads = grad(model, batch_x, batch_y)
        optimizer.apply_gradients(zip(grads, model.variables), global_step=tf.train.get_or_create_global_step())

    train_loss = lossfun(model, train_x_tf, train_y_tf)
    correct_prediction = tf.equal(tf.argmax(model(train_x_tf), axis=1, output_type=tf.int32), train_y_tf)
    train_acc = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    valid_loss = lossfun(model, valid_x_tf, valid_y_tf)
    correct_prediction = tf.equal(tf.argmax(model(valid_x_tf), axis=1, output_type=tf.int32), valid_y_tf)
    valid_acc = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        
    if (epoch+1)%5 == 0:
        tqdm.write('epoch:\t{}\ttrain/loss:\t{:.5f}\tvalid/loss:\t{:.5f}\ttrain/accuracy:\t{:.5f}\tvalid/accuracy:\t{:.5f}'.format(
            epoch+1, train_loss, valid_loss, train_acc, valid_acc)
                  )

epoch:	5	train/loss:	0.97823	valid/loss:	1.01635	train/accuracy:	0.35714	valid/accuracy:	0.26316
epoch:	10	train/loss:	0.76805	valid/loss:	0.75222	train/accuracy:	0.76786	valid/accuracy:	0.81579
epoch:	15	train/loss:	0.62252	valid/loss:	0.58292	train/accuracy:	0.89286	valid/accuracy:	0.92105
epoch:	20	train/loss:	0.52273	valid/loss:	0.46825	train/accuracy:	0.89286	valid/accuracy:	0.97368
epoch:	25	train/loss:	0.45645	valid/loss:	0.39486	train/accuracy:	0.89286	valid/accuracy:	0.97368
epoch:	30	train/loss:	0.40661	valid/loss:	0.34131	train/accuracy:	0.92857	valid/accuracy:	0.97368
100%|██████████| 30/30 [00:01<00:00, 17.36it/s]


* 高レベルAPIの使い方+Eager だと, やはり高レベルなライブラリで Define by Run の Chainer や PyTorch に似てくる.
* 以下のように, Pythonで動的グラフを制御して学習させることが可能に.

In [9]:
class Model(tfe.Network):
    
    def __init__(self):
        super(Model, self).__init__()
        input_size = 4
        output_size = 3
        hidden_size = 20
        self.fc1 = self.track_layer(tf.layers.Dense(hidden_size, input_shape=(input_size, )))
        self.fc2 = self.track_layer(tf.layers.Dense(hidden_size, input_shape=(hidden_size, )))
        
        self.fc2_2 = self.track_layer(tf.layers.Dense(hidden_size, input_shape=(hidden_size, ))) # もう一つ無駄に順伝播作って
        
        self.fc3 = self.track_layer(tf.layers.Dense(output_size, input_shape=(hidden_size, )))
    
    def __call__(self, x):
        
        h = tf.nn.relu(self.fc1(x))
        h = tf.nn.relu(self.fc2(h))
        
        # ランダムにもう一つ無駄に通すという意味のない分岐をするネットワーク
        prob = np.random.randn()
        if prob > 0:
            h = tf.nn.relu(self.fc2_2(h))
            
        y = self.fc3(h)
        return y
    
model = Model()

def lossfun(model, x, y):
    y_pre = model(x)
    y_oh = tf.one_hot(y, depth=output_size, dtype=tf.float32)
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_oh, logits=y_pre))
    return cross_entropy

def grad(model, x, y):
    with tf.GradientTape() as tape:
        loss = lossfun(model, x, y)
    return tape.gradient(loss, model.variables)
    
train_x_tf = tf.convert_to_tensor(train_x, dtype=tf.float32)
train_y_tf = tf.convert_to_tensor(train_y, dtype=tf.int32)
valid_x_tf = tf.convert_to_tensor(valid_x, dtype=tf.float32)
valid_y_tf = tf.convert_to_tensor(valid_y, dtype=tf.int32)

epoch_num = 30
batch_size = 16

optimizer = tf.train.AdamOptimizer()

for epoch in tqdm(range(epoch_num), file=sys.stdout):
    
    n, _ = train_x_tf.shape
    n = n.value
    perm = np.random.permutation(n)
    
    for i in range(0, n, batch_size):

        batch_x = tf.gather(train_x_tf, perm[i:i+batch_size])
        batch_y = tf.gather(train_y_tf, perm[i:i+batch_size])

        grads = grad(model, batch_x, batch_y)
        optimizer.apply_gradients(zip(grads, model.variables), global_step=tf.train.get_or_create_global_step())

    train_loss = lossfun(model, train_x_tf, train_y_tf)
    correct_prediction = tf.equal(tf.argmax(model(train_x_tf), axis=1, output_type=tf.int32), train_y_tf)
    train_acc = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    valid_loss = lossfun(model, valid_x_tf, valid_y_tf)
    correct_prediction = tf.equal(tf.argmax(model(valid_x_tf), axis=1, output_type=tf.int32), valid_y_tf)
    valid_acc = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        
    if (epoch+1)%5 == 0:
        tqdm.write('epoch:\t{}\ttrain/loss:\t{:.5f}\tvalid/loss:\t{:.5f}\ttrain/accuracy:\t{:.5f}\tvalid/accuracy:\t{:.5f}'.format(
            epoch+1, train_loss, valid_loss, train_acc, valid_acc)
                  )

epoch:	5	train/loss:	0.93633	valid/loss:	0.94247	train/accuracy:	0.50000	valid/accuracy:	0.31579
epoch:	10	train/loss:	0.82767	valid/loss:	0.81756	train/accuracy:	0.69643	valid/accuracy:	0.71053
epoch:	15	train/loss:	0.71850	valid/loss:	0.70159	train/accuracy:	0.72321	valid/accuracy:	0.81579
epoch:	20	train/loss:	0.46483	valid/loss:	0.40825	train/accuracy:	0.86607	valid/accuracy:	0.97368
epoch:	25	train/loss:	0.39936	valid/loss:	0.33973	train/accuracy:	0.91071	valid/accuracy:	0.97368
epoch:	30	train/loss:	0.46135	valid/loss:	0.28049	train/accuracy:	0.90179	valid/accuracy:	0.97368
100%|██████████| 30/30 [00:01<00:00, 15.51it/s]


* 通常モードでGPUを使う場合は明示的に宣言する必要はなかったが, EagerモードでGPUで使う場合は以下のように使い分けて宣言する必要がある

In [ ]:
# GPU上に値を持たせる場合
with tf.device('/gpu:0'):
    x = tf.random_normal([10, 10])
    
# CPU上に値を持たせる場合
with tf.device('/cpu:0'):
    x = tf.random_normal([10, 10])
    
# 何も指定しない場合はCPU上に持つ
x = tf.random_normal([10, 10])

# CPUからGPUへ
with tf.device('/cpu:0'):
    x = tf.random_normal([10, 10])
x_gpu = x.gpu()

# GPUからCPUへ
with tf.device('/gpu:0'):
    x = tf.random_normal([10, 10])
x_cpu = x.cpu()

**感想**

* Chainer, PyTorchと比べて特に秀でた要素は見当たらなかったので, Chainer, PyTorch, TensorFlow, TensorFlow Eagerどれでも好きなのを使えば良いかなという印象. (TPUが使えるくらい？）
* 結構numpyライクに書けるように？これをGPUで行えるようになると考えると, 機械学習に限らず, 一般的な計算でも便利になる場面があるかも？（データを取り込むところからTensorFlowだけで完結させたら, どう異なるか試したい）
* 高レベルAPIで使うか, 低レベルAPIで使うか, 先に決めておかないとTensorFlow死ねる.